In [ ]:
import requests
import time
from datetime import datetime
import pymysql
from tqdm import tqdm
import pickle

# 스팀 BaseID 부터 i를 증가시킨 아이디를 반환 (i가 1이상일때, 제대로 작동)
def baseID(i):
    baseID = 76561197960265728
    baseID += i
    str_ID = str(baseID)
    return str_ID


# 스팀 유저의 게임 라이브러리를 가져오는 함수
# name, appid(game id로 추정), playtime_forever(총 게임 플레이 시간)을 list of dictionary로 가지고 옴
def get_user_game_library(steam_api_key, steam_user_id):
    url = "http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/"
    params = {
        "key": steam_api_key,
        "steamid": steam_user_id,
        "format": "json",
        "include_appinfo": True,
        "include_played_free_games": True,
    }
    response = requests.get(url, params=params)
    # 조회가 성공한 경우 status_code == 200
    if response.status_code == 200:
        data = response.json()
        games = data.get("response", {}).get("games", [])
        # 유저 아이디가 비어 있는 경우 비어있는 list ([])를 반환
        return [
            {
                "name": game.get("name", "nan"),
                "appid": game.get("appid", "nan"),
                "playtime_forever": game.get("playtime_forever", 0),
                "playtime_2weeks": game.get("playtime_2weeks", 0),
                "img_icon_url": game.get("img_icon_url", "nan")
            }
            for game in games
        ]
    # 조회가 실패한 경우 알림을 띄우고 -1 반환
    else:
        print("Failed to retrieve data.")
        return -1
    
# 스팀 유저의 정보를 가져오는 함수  
def get_user_summary(steam_api_key, steam_user_id):
    url = "http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/"
    params = {
        "key": steam_api_key,
        "steamids": steam_user_id,
        "format": "json"
    }
    response = requests.get(url, params=params)
    # 조회가 성공한 경우 status_code == 200
    if response.status_code == 200:
        data = response.json()
        players = data.get("response", {}).get("players", [])
        # 유저 아이디가 비어 있는 경우 비어있는 list ([])를 반환
        return [
            {
                "personaname": player.get("personaname", "nan"),
                "data_time": datetime.now()
                
            }
            for player in players
        ]
    # 조회가 실패한 경우 알림을 띄우고 -1 반환
    else:
        print("Failed to retrieve data.")
        return -1

# 스팀 게임의 추가 정보를 가져오는 함수     
def get_app_add_info(app_id):
    url = "https://store.steampowered.com/api/appdetails"
    params = {
        "appids": app_id
    }
    response = requests.get(url, params=params)
    # 조회가 성공한 경우 status_code == 200
    if response.status_code == 200:
        data = response.json()
        items = data.get(app_id, {}).get("data", [])
        # 유저 아이디가 비어 있는 경우 비어있는 list ([])를 반환
        return items
#         return [
#             {
#                 #사용할 FEATURE 목록 DICT 형식으로 추가
                
#             }
#             for item in items
#         ]
    # 조회가 실패한 경우 알림을 띄우고 -1 반환
    else:
        print("Failed to retrieve data.")
        return -1

In [ ]:
# API key를 저장한 txt 파일
file_path = "api_key.txt"

with open(file_path, "r", encoding="utf-8") as file:
    steam_api_key = file.read()

In [ ]:
# user list를 담을 dictionary 생성
user_game_list = {}
user_profile = {}

for i in tqdm(range(1, 10000)):
    temp_dict = get_user_game_library(steam_api_key, baseID(i))
    
    if temp_dict == -1:  # 조회가 실패한 경우 반복문 continue 하는 것으로 수정
        continue
    else: # 조회가 성공한 경우
        time.sleep(1) # API 호출을 위한 속도 조절
        if temp_dict:
            user_game_list[baseID(i)] = temp_dict
            
            # 게임 정보가 있는 유저에 한해서 유저 정보 가져오기
            temp_dict = get_user_summary(steam_api_key, baseID(i))
            if temp_dict == -1:
                continue
            else:
                time.sleep(1) # API 호출을 위한 속도 조절
                if temp_dict:
                    user_profile[baseID(i)] = temp_dict
            

In [ ]:
# 가져온 유저 개수 확인
len(user_list)

In [ ]:
with open('user_games_1-10000.pickle','wb') as fw:
    pickle.dump(user_list, fw)

In [ ]:
with open('user_profile_1-10000.pickle','wb') as fw:
    pickle.dump(user_profile, fw)